In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-OGTOHVK:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1604937783486)
SparkSession available as 'spark'


import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD


## Class definition

In [22]:
class Shingle {
    
    def createshingles(path: String, k:Int) : RDD[(String, Set[Int])] = {
        val files = sc.wholeTextFiles(path,1).map({ case (path, text) => (path.split("/").last, text) })
                                            .mapValues(_.toLowerCase)
                                            .mapValues(_.replace(",", ""))
                                            .mapValues(_.replace(".", ""))
                                            .mapValues(_.replace("?", ""))
                                            .mapValues(_.replace("!", ""))
                                            .mapValues(_.replace(")", ""))
                                            .mapValues(_.replace("(", ""))    
        val shingles = files.flatMapValues { case (str) => str.sliding(k,1)
        }  
        print(shingles.collect)
        return shingles.mapValues(str => str.hashCode()).groupByKey().mapValues(x => x.toSet)
    }
}


class CompareSet {
  def JaccardSimilarity(set1: Set[(Int)], set2: Set[(Int)]) : Double = {
    return ((set1.intersect(set2).size).toDouble / (set1.union(set2).size)).toDouble
  }
}

implicit def bool2int(b:Boolean) = if (b) 1 else 0
import scala.util.control._

class MinHashing {
    
  def create_chrMatrix(shingles_set: Array[(Int)], set: Array[(Int)]) : Array[(Int)] = {
    val chr_col = Array.fill(shingles_set.size)(0)
      for (i <- 0 to shingles_set.size-1){
        chr_col(i) = (set contains shingles_set(i))*1
      }
    return chr_col
  }
    
  def h(x: Int, a:Int, rows:Int) : Int =  {
        return (((x+1)*a)+1)%rows
    }

  def create_signature(shingles_set: Array[(Int)], set: Array[(Int)], permutations: Int) : Array[(Int)] = {
      val chr_col = create_chrMatrix(shingles_set, set)
      val signature = Array.fill(permutations)(0)
      val rows = chr_col.size
      val inloop = new Breaks
          for (a <- 1 to permutations){
              inloop.breakable{
              for (x <- 0 to chr_col.size-1){
                  if (chr_col(h(x,a,rows))==1){
                      signature(a-1) = x+1
                      inloop.break
                      }
                  }
              }
          }
      return signature
  }
    
}

class CompareSignatures {
    
   def similarity(signature0: Array[(Int)], signature1: Array[(Int)]) : Double = {
       val sign0 = signature0.toList
       val sign1 = signature1.toList
       return (sign0.zip(sign1).filter(x => x._1 == x._2).size).toDouble/signature0.size.toDouble
    } 
}

import scala.collection.mutable.ListBuffer

class LocSensHashing{
    
    def candidate_pairs(signatures: Vector[(String, Array[Int])], r: Int) : Vector[(String, String)] = {
        
        val loc_hash = for (i <- 0 to signatures.size-1) yield((signatures(i)._1,
                                                (for (x <- (signatures(i)._2.toList).grouped(r)) yield(x.hashCode())).toList))
        
        val candidates = new ListBuffer[(String, String)]()
        
        for (i <- 0 to signatures.size-1){
            for (j <- i+1 to signatures.size-1){
               if ((loc_hash(i)._2.zip(loc_hash(j)._2).filter(x => x._1 == x._2).size) > 0){
                   candidates += ((signatures(i)._1, signatures(j)._1))
                   //println(signatures(i)._1, signatures(j)._1)
               }
            }
        }
        return candidates.toVector
    }
}

defined class Shingle
defined class CompareSet
bool2int: (b: Boolean)Int
import scala.util.control._
defined class MinHashing
defined class CompareSignatures
import scala.collection.mutable.ListBuffer
defined class LocSensHashing


---

In [15]:
// parameters

val path = "../data/*_taska.txt"
val k = 7

path: String = ../data/*_taska.txt
k: Int = 7


In [16]:
// Create shingles

val shing = new Shingle
val shingles = shing.createshingles(path,k)
val s = shingles.collect()

[Lscala.Tuple2;@34cf176f

shing: Shingle = Shingle@69536633
shingles: org.apache.spark.rdd.RDD[(String, Set[Int])] = MapPartitionsRDD[27] at mapValues at <console>:45
s: Array[(String, Set[Int])] = Array((g0pE_taska.txt,Set(1024328849, -809942811, 300626, 450333591, -1110351275, 2057428624, 1237394832, -719776277, 877982873, -1509224606, -1348893292, -422012319, 1772178932, -974723696, -604869464, 1718732402, -1897320720, 2129320190, 1217550084, -1351981020, -1542092980, -1956689524, 1979091318, -679719782, -887312734, 667020100, 254387060, 735439297, 1219629894, 1930547162, 1195890950, -1922170827, 833328706, -1911224770, 700148195, -493883412, -1433668758, -1494609322, -1191026030, 1361461549, -1311872163, 1606945631, 939821247, 1216975005, 254386680, 1020649433, 280184718, -77600629, -421994171, 1998498434,...


In [17]:
// Compute similarity

val similarity = new CompareSet
val set_size = s.size-1

for (i <- 0 to set_size){
    for (j <- i+1 to set_size){
        val sim = similarity.JaccardSimilarity(s(i)._2,s(j)._2)
        println("("+s(i)._1+","+s(j)._1+"):"+sim)
    }
}

(g0pE_taska.txt,g0pC_taska.txt):0.15293080652269722
(g0pE_taska.txt,g1pA_taska.txt):0.05274000824062629
(g0pE_taska.txt,g3pC_taska.txt):0.37671641791044774
(g0pE_taska.txt,g0pD_taska.txt):0.5091230135373749
(g0pE_taska.txt,g0pB_taska.txt):0.04813432835820895
(g0pE_taska.txt,g4pD_taska.txt):0.22662367572547212
(g0pE_taska.txt,g3pA_taska.txt):0.06424344885883347
(g0pE_taska.txt,g1pB_taska.txt):0.04103671706263499
(g0pE_taska.txt,g4pC_taska.txt):0.8371958285052143
(g0pE_taska.txt,g2pE_taska.txt):0.5117101193106496
(g0pE_taska.txt,g2pA_taska.txt):0.03694581280788178
(g0pE_taska.txt,g2pB_taska.txt):0.03636363636363636
(g0pE_taska.txt,orig_taska.txt):0.8897228637413395
(g0pE_taska.txt,g4pE_taska.txt):0.09837467921300257
(g0pE_taska.txt,g0pA_taska.txt):0.05443548387096774
(g0pE_taska.txt,g2pC_taska.txt):0.36591606960081885
(g0pE_taska.txt,g3pB_taska.txt):0.05779889152810768
(g0pE_taska.txt,g4pB_taska.txt):0.03252710592160134
(g0pE_taska.txt,g1pD_taska.txt):0.23805309734513275
(g0pC_taska.txt,

(g2pA_taska.txt,g2pC_taska.txt):0.037695891571368066
(g2pA_taska.txt,g3pB_taska.txt):0.06190476190476191
(g2pA_taska.txt,g4pB_taska.txt):0.041051660516605165
(g2pA_taska.txt,g1pD_taska.txt):0.04202020202020202
(g2pB_taska.txt,orig_taska.txt):0.036458333333333336
(g2pB_taska.txt,g4pE_taska.txt):0.0431077694235589
(g2pB_taska.txt,g0pA_taska.txt):0.08295165394402036
(g2pB_taska.txt,g2pC_taska.txt):0.04351984696317551
(g2pB_taska.txt,g3pB_taska.txt):0.06429615197272284
(g2pB_taska.txt,g4pB_taska.txt):0.08274360370168754
(g2pB_taska.txt,g1pD_taska.txt):0.038185085354896675
(orig_taska.txt,g4pE_taska.txt):0.09406952965235174
(orig_taska.txt,g0pA_taska.txt):0.05259087393658159
(orig_taska.txt,g2pC_taska.txt):0.35546875
(orig_taska.txt,g3pB_taska.txt):0.05705591479650057
(orig_taska.txt,g4pB_taska.txt):0.034441329595514616
(orig_taska.txt,g1pD_taska.txt):0.22677364864864866
(g4pE_taska.txt,g0pA_taska.txt):0.04833679833679834
(g4pE_taska.txt,g2pC_taska.txt):0.046488125315816066
(g4pE_taska.txt,

similarity: CompareSet = CompareSet@5c648af9
set_size: Int = 19


In [18]:
//MinHashing

// create common set of shingles
val shingles_list = scala.util.Random.shuffle(
                    ((for (x <- 0 to set_size) yield (s(x)._2).toList).flatten).distinct).toArray

// create the signature matrix
val minhash = new MinHashing
val permutations = 50

val sign_list = (for (x <- 0 to set_size) 
                 yield ((s(x)_1,minhash.create_signature(shingles_list, (s(x)_2).toArray, permutations))))

shingles_list: Array[Int] = Array(-1606743939, 1215015886, -437163181, -434740446, -1609047663, -978674325, -1906396082, 70425113, -1237467926, 202479, 1552544442, 362001712, -1536747481, 1020538543, -1208241579, -752681339, -1857225274, 1097725878, -1478064976, -852967472, -1554937767, 460120858, 1375754130, 884961328, -2100160097, -347567283, -981816313, 280184718, -1535362694, -678739246, -1480412341, 2137231266, 1294676549, 1886248921, -1907248098, 1161847312, -1146159272, 1765258938, -2032854607, 1421108136, 9282342, 914054319, 1084750433, -1591967590, -594389885, -140520262, -1336269169, -1313720472, 1835436030, -752694787, -291571261, 1199671284, -710603304, 1957584561, -1546824622, -1857213676, 463460819, 756371877, -911661498, 1038297195, 13159815, -368958482, 1206420497, 15165...


In [19]:
val minhash = new MinHashing
val permutations = 50

val sign_list = (for (x <- 0 to set_size) 
                 yield ((s(x)_1,minhash.create_signature(shingles_list, (s(x)_2).toArray, permutations))))

minhash: MinHashing = MinHashing@1fe86d75
permutations: Int = 50
sign_list: scala.collection.immutable.IndexedSeq[(String, Array[Int])] = Vector((g0pE_taska.txt,Array(7, 9, 5, 5, 3, 3, 1, 3, 2, 2, 3, 2, 2, 3, 1, 4, 9, 1, 4, 1, 2, 3, 9, 1, 7, 1, 26, 11, 3, 1, 3, 2, 1, 9, 2, 19, 14, 2, 5, 6, 2, 1, 4, 4, 7, 24, 13, 5, 7, 6)), (g0pC_taska.txt,Array(3, 13, 1, 9, 3, 6, 6, 28, 4, 9, 20, 3, 2, 3, 1, 14, 14, 2, 7, 11, 2, 10, 9, 15, 7, 1, 26, 8, 18, 3, 1, 7, 13, 7, 5, 1, 23, 7, 10, 9, 11, 1, 4, 5, 2, 6, 1, 20, 5, 6)), (g1pA_taska.txt,Array(1, 10, 14, 5, 4, 7, 6, 4, 6, 2, 5, 8, 7, 3, 13, 2, 11, 3, 10, 1, 2, 7, 45, 4, 32, 29, 2, 4, 6, 7, 11, 1, 3, 45, 6, 45, 24, 5, 5, 4, 8, 1, 16, 21, 23, 28, 13, 2, 10, 16)), (g3pC_taska.txt,Array(10, 5, 5, 5, 2, 3, 6, 8, 2, 1, 20, 57, 7, 3, 1, 4, 13, 1, 36, 1, 2...


In [20]:
// Signatures comparison

val compSign = new CompareSignatures

for (i <- 0 to set_size){
    for (j <- i+1 to set_size){
        val sim = compSign.similarity(sign_list(i)._2,sign_list(j)._2)
        println("("+sign_list(i)._1+","+sign_list(j)._1+"):"+sim)
    }
}

(g0pE_taska.txt,g0pC_taska.txt):0.24
(g0pE_taska.txt,g1pA_taska.txt):0.16
(g0pE_taska.txt,g3pC_taska.txt):0.32
(g0pE_taska.txt,g0pD_taska.txt):0.5
(g0pE_taska.txt,g0pB_taska.txt):0.12
(g0pE_taska.txt,g4pD_taska.txt):0.18
(g0pE_taska.txt,g3pA_taska.txt):0.16
(g0pE_taska.txt,g1pB_taska.txt):0.06
(g0pE_taska.txt,g4pC_taska.txt):0.88
(g0pE_taska.txt,g2pE_taska.txt):0.34
(g0pE_taska.txt,g2pA_taska.txt):0.02
(g0pE_taska.txt,g2pB_taska.txt):0.0
(g0pE_taska.txt,orig_taska.txt):0.9
(g0pE_taska.txt,g4pE_taska.txt):0.16
(g0pE_taska.txt,g0pA_taska.txt):0.18
(g0pE_taska.txt,g2pC_taska.txt):0.44
(g0pE_taska.txt,g3pB_taska.txt):0.14
(g0pE_taska.txt,g4pB_taska.txt):0.06
(g0pE_taska.txt,g1pD_taska.txt):0.22
(g0pC_taska.txt,g1pA_taska.txt):0.08
(g0pC_taska.txt,g3pC_taska.txt):0.16
(g0pC_taska.txt,g0pD_taska.txt):0.14
(g0pC_taska.txt,g0pB_taska.txt):0.06
(g0pC_taska.txt,g4pD_taska.txt):0.02
(g0pC_taska.txt,g3pA_taska.txt):0.1
(g0pC_taska.txt,g1pB_taska.txt):0.06
(g0pC_taska.txt,g4pC_taska.txt):0.26
(g0pC

compSign: CompareSignatures = CompareSignatures@2841ae29


In [32]:
// Local-Sensitive Hashing

val t = 0.8
val b = 5
val r = 10

val LSH = new LocSensHashing
val cand = LSH.candidate_pairs(sign_list.toVector, r);

t: Double = 0.8
b: Int = 5
r: Int = 10
LSH: LocSensHashing = LocSensHashing@36393bbb
cand: Vector[(String, String)] = Vector((g0pE_taska.txt,g0pD_taska.txt), (g0pE_taska.txt,g4pC_taska.txt), (g0pE_taska.txt,orig_taska.txt), (g0pD_taska.txt,g4pC_taska.txt), (g0pD_taska.txt,orig_taska.txt), (g4pC_taska.txt,orig_taska.txt))
